In [1]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *
import matplotlib.pyplot as plt
from glob import glob
import os
from skimage.io import imread, imsave
from time import time

### Get file paths

In [2]:
folder_path = r"D:\Magnumfe\al6-1-1_peem_profile2_10mT15deg"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
raytracing_file = os.path.join(data_folder, "raytracing.npy")

# sort mag files
# indx = [int(mf.split('.')[-2]) for mf in mag_files]
indx = [int(mf.split('_')[-1].split('.')[0]) for mf in mag_files]
mag_files = [mf for _, mf in sorted(zip(indx, mag_files))]

## If raytracing generated

In [ ]:
# load raytracing if exists
raytr = np.load(raytracing_file, allow_pickle=True).item()
struct = raytr.struct
struct_projected = raytr.struct_projected

## Generate raytracing

In [4]:
# need to have a mesh file
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]
# get the mesh
msh = Mesh.from_file(msh_file)

#### Make sure that the projection vector is correct and that the structure is oriented well

In [5]:
# get the projection vector
p = get_projection_vector(90, 16)

# prepare raytracing object
raytr = RayTracing(msh, p)
struct = raytr.struct
struct_projected = raytr.struct_projected

In [7]:
vis = MeshVisualizer(struct, struct_projected)
vis.show()

#### Get the ray tracing information and save

In [6]:
t0 = time()
raytr.get_piercings()
np.save(raytracing_file, raytr, allow_pickle=True)
print(time() - t0)

 29%|███████                 | 32049/109392 [03:53<09:22, 137.59it/s]D:\Programs\xmcd-projection\xmcd_projection\raytracing.py:80: UserWarning: Wrong number of intersections! Ensure that tetrahedra are valid and that the projection plane does not intersect the structure. If this happens rarely, it could be a numerical artefact.
  except ValueError as e:
100%|███████████████████████| 109392/109392 [13:17<00:00, 137.21it/s]


829.9110107421875


### Get the xmcd

#### Get the magnetisation from csv files
Note: sometimes if the mesh file has multiple parts, the paraview export and the original mesh coordinates are not in the same order. I add a function to fix that when necessary

In [7]:
magnetisation, mag_points = load_mesh_magnetisation(mag_files[22])
shuffle_file = os.path.join(data_folder, "shuffle_indx.npy")
try:
    shuffle_indx = np.load(shuffle_file)
except FileNotFoundError:
    print('File not found. Generating shuffle indx')
    shuffle_indx = msh.get_shuffle_indx(mag_points)
    np.save(shuffle_file, shuffle_indx)
magnetisation = magnetisation[shuffle_indx, :]

### Get the colours and XMCD values

In [8]:
xmcd_value = raytr.get_xmcd(magnetisation)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

In [10]:

deep_reload(xmcd_projection)
from xmcd_projection import *
center_struct = [-100, 100, 0]
dist_struct = 1e5
center_peem = [600, -5000, 0]
dist_peem = 7e5

vis = MeshVisualizer(struct, struct_projected, projected_xmcd=xmcd_value, struct_colors=mag_colors)
vis.show(azi=18, center=center_peem, dist=dist_peem)

#### View different parts of the image separately

#### Both

In [15]:
vis.update_colors(xmcd_value, mag_colors)
vis.view_both(azi=18, center=center_peem, dist=dist_peem)

#### Projection

In [14]:
vis.view_projection(azi=18, center=center_peem, dist=dist_peem)

#### Structure

In [13]:
vis.view_struct(azi=18, center=center_struct, dist=dist_struct)

#### Saving one render

In [110]:
vis.view_both(azi=18, center=center_peem, dist=dist_peem)
vis.save_render(os.path.join(folder_path, 'processed', 'shadow_final.png'))
vis.view_projection()
blurred = vis.get_blurred_image(desired_background=0.7)
imsave(os.path.join(folder_path, 'processed', 'shadow_final_blurred.png'), (blurred*255).astype(np.uint8), check_contrast=False)

vis.view_struct(azi=18, center=center_struct, dist=dist_struct)
vis.save_render(os.path.join(folder_path, 'processed', 'structure_view.png'))

### Run for all magnetisations and save images

In [87]:
desired_background = 0.7 # if not None, adjusts the contrast so that the background is of the desired value in the blurred image

# define save directories
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection')
savedir_blurred = savedir+'_blurred1'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)

# run and save
for mf in mag_files:
    file_num = int(mf.split('.')[-2])
#     file_num = int(mf.split('_')[-1].split('.')[0])
    # get the magnetisation and xmcd
    magnetisation, _ = get_mesh_magnetisation(mf)
    magnetisation = magnetisation[shuffle_indx, :]
    xmcd_value = raytr.get_xmcd(magnetisation)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
    vis.view_projection(azi=18, center=center_peem, dist=dist_peem)
    file_name = 'step_{}.png'.format(file_num)
    image = vis.save_render(os.path.join(savedir, file_name))
    
    # save the blurred image
    blurred = img2uint(vis.get_blurred_image(desired_background=desired_background))
    imsave(os.path.join(savedir_blurred, file_name), blurred, check_contrast=False)
    
    # save the image of the structure
    vis.view_struct(azi=18, center=center_struct, dist=dist_struct)
    image = vis.save_render(os.path.join(savedir_structure, file_name))

In [207]:
desired_background = 0.7 # if not None, adjusts the contrast so that the background is of the desired value in the blurred image

savedir = os.path.join(folder_path, 'processed', 'xmcd_projection_inv')
savedir_blurred = savedir+'_blurred1'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure_inv')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)
        
for mf in mag_files:
#     file_num = int(mf.split('.')[-2])
    file_num = int(mf.split('_')[-1].split('.')[0])
    magnetisation, _ = get_magnumfe_magnetisation(mf)
    magnetisation *= -1
    magnetisation = magnetisation[shuffle_indx, :]
    xmcd_value = get_xmcd_from_piercings_list(tetra, magnetisation, piercings_list, p=p)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
    vis.view_projection(azi=18, center=center_peem, dist=dist_peem)
    file_name = 'step_{}.png'.format(file_num)
    image = vis.save_render(os.path.join(savedir, file_name))
    
    # save the blurred image
    blurred = img2uint(vis.get_blurred_image(desired_background=desired_background))
    imsave(os.path.join(savedir_blurred, file_name), blurred, check_contrast=False)
    
    # save the image of the structure
    vis.view_struct(azi=18, center=center_struct, dist=dist_struct)
    image = vis.save_render(os.path.join(savedir_structure, file_name))


### Import the saved images and make a movie

In [90]:
import cv2
from tqdm import tqdm
FPS = 10

out_file = os.path.join(folder_path, 'processed', 'projection_movie_inv.mp4')
image_files = glob(os.path.join(savedir, '*.png'))

indx = [int(imgf.split('.')[-2].split('_')[-1]) for imgf in image_files]
image_files = [imgf for _, imgf in sorted(zip(indx, image_files))]

# Define the codec and create VideoWriter object
fourcc =cv2.VideoWriter_fourcc('M','J','P','G') # Be sure to use lower case
img = cv2.imread(image_files[0])
out_writer = cv2.VideoWriter(out_file, fourcc, FPS, (img.shape[1], img.shape[0]))

i = 0
for imf in tqdm(image_files):
    img = cv2.imread(imf)
    # append to the video writer
    out_writer.write(img)
    
# Release everything if job is finished
out_writer.release()
cv2.destroyAllWindows()

print("The output video is {}".format(out_file))

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 53.92it/s]

The output video is F:\PhD_data\Simulations\Alba_spirals\scaled\al6-1-1_asbuiltnotop_scaled\processed\projection_movie_inv.mp4


# Angles study

In [65]:
folder_path = r"D:\Magnumfe\angles_initialisation"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]

mesh = get_mesh(msh_file)
struct = get_struct_from_mesh(mesh)
p = get_projection_vector(90, 16)
projected_structure = project_structure_byvector(struct, p=p)

magnetisation, _ = get_magnumfe_magnetisation(mag_files[-1])
xmcd_value = get_xmcd_from_piercings_list(mesh, magnetisation, piercings_list)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

In [66]:
vis = MagnumfeVisualizer(struct, struct_projected, xmcd_value, struct_colors=mag_colors)
vis.show()
vis.set_camera(azi=18, center=vis.get_structs_center(), dist=6e5)

In [60]:
os.path.basename(mag_files[0]).split('.')[0]

'al6-1-1_peem_angle-15'

In [ ]:
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection')
if not os.path.exists(savedir):
    os.makedirs(savedir)
    
for mf in tqdm(mag_files):
    save_name = os.path.basename(mf).split('.')[0]+'.png'
    save_path = os.path.join(savedir, save_name)
    if os.path.exists(save_path):
        print('continuing')
        continue
    magnetisation, _ = get_magnumfe_magnetisation(mf)
#     magnetisation *= -1
    xmcd_value = get_xmcd_from_piercings_list(mesh, magnetisation, piercings_list)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    vis.update_colors(xmcd_value, mag_colors)
    image = vis.save_render(save_path)

In [70]:
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection_inv')
if not os.path.exists(savedir):
    os.makedirs(savedir)
    
for mf in tqdm(mag_files):
    save_name = os.path.basename(mf).split('.')[0]+'.png'
    save_path = os.path.join(savedir, save_name)
    if os.path.exists(save_path):
        print('continuing')
        continue
    magnetisation, _ = get_magnumfe_magnetisation(mf)
    magnetisation *= -1
    xmcd_value = get_xmcd_from_piercings_list(mesh, magnetisation, piercings_list)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    vis.update_colors(xmcd_value, mag_colors)
    image = vis.save_render(save_path)



  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

continuing




  7%|█████▉                                                                             | 1/14 [00:00<00:01,  8.20it/s]

continuing
continuing
continuing
continuing




 43%|███████████████████████████████████▌                                               | 6/14 [00:13<00:07,  1.11it/s]

continuing
continuing




 64%|█████████████████████████████████████████████████████▎                             | 9/14 [00:27<00:09,  1.98s/it]

continuing
continuing
continuing
continuing
continuing




100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:27<00:00,  1.95s/it]

In [71]:
folder_path

'D:\\Magnumfe\\angles_initialisation'